# Spark SQL 

In [1]:
from pyspark.sql import SparkSession

# Enable Hive sync with spark dataframe session

In [2]:
spark_session = SparkSession.builder.enableHiveSupport().appName("spark Sql").master("local").getOrCreate()

# Show tables in Hive DB  
* Using Spark Session

In [3]:
dataframe = spark_session.catalog.listDatabases()
print(dataframe)

[Database(name='default', description='Default Hive database', locationUri='hdfs://localhost:9000/user/hive/warehouse'), Database(name='demodb', description='', locationUri='hdfs://localhost:9000/user/brainster/test_metastore'), Database(name='own_database', description='', locationUri='hdfs://localhost:9000/user/jovyan/database_metastore'), Database(name='stackoverflow_', description='', locationUri='hdfs://localhost:9000/user/jovyan/database_metastore'), Database(name='test', description='', locationUri='file:/home/jovyan/spark-warehouse/test.db')]


In [4]:
listTables = spark_session.catalog.listTables("default")
print(listTables)

[Table(name='employee', database='default', description='Employee details', tableType='MANAGED', isTemporary=False), Table(name='employee2', database='default', description='Employee details', tableType='MANAGED', isTemporary=False), Table(name='user_activity', database='default', description='User Activity', tableType='MANAGED', isTemporary=False)]


# Get databases info

In [5]:
spark_session.sql("""show tables in default""").toPandas()

,database,tableName,isTemporary
0,default,employee,False
1,default,employee2,False
2,default,user_activity,False


# Show user_activity table info

In [6]:
spark_session.sql("""describe default.user_activity""").toPandas()

,col_name,data_type,comment
0,first_name,string,None
1,last_name,string,None
2,email,string,None
3,gender,string,None
4,ip_adress,string,None
5,company_name,string,None
6,purchase_ammount,int,None
7,job_title,string,None
8,user_agent,string,None


# Read Table into Spark Dataframe (distributed collection of data)

In [7]:
dataframe = spark_session.read.table("default.user_activity")

dataframe.toPandas().head()

,first_name,last_name,email,gender,ip_adress,company_name,purchase_ammount,job_title,user_agent
0,Amandi,Gibbetts,agibbetts0@live.com,Female,39.231.192.239,Brainlounge,81,JCR,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8..."
1,Mathew,Michin,mmichin1@bloglovin.com,Male,115.35.31.40,Mydo,789,Dell KACE,"""Mozilla/5.0 (Macintosh; U; PPC Mac OS X 10_5_..."
2,Omero,Petrina,opetrina2@ycombinator.com,Male,42.43.198.154,Agivu,703,HP Service Manager,"""Mozilla/5.0 (X11; Linux i686) AppleWebKit/535..."
3,Clevey,Jesse,cjesse3@reverbnation.com,Male,99.233.153.203,Chatterbridge,740,Bioinformatics,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
4,Wit,Gent,wgent4@earthlink.net,Male,235.73.119.127,Zoomlounge,657,Procurement Outsourcing,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537...."


# Query 1 :

Show total purchase ammount sum (Income)

In [8]:
import pyspark.sql.functions as f

In [9]:
purchase_sum = dataframe.groupBy(dataframe.company_name)\
.agg(f.sum(dataframe.purchase_ammount).alias("sum_purchase"))\
.orderBy("sum_purchase", ascending=False).limit(5)
purchase_sum.toPandas()

,company_name,sum_purchase
0,Katz,14383
1,Livetube,11823
2,Skimia,10171
3,Thoughtstorm,9763
4,Quatz,9580


In [10]:
purchase_sum = dataframe.agg(f.sum(dataframe.purchase_ammount).alias("sum_purchase"))\
.orderBy("sum_purchase", ascending=False).limit(5)
purchase_sum.toPandas()

,sum_purchase
0,1580183


# Query 2 : 

Count distinct First Name users from the dataset

In [11]:
distinct_names = dataframe.groupBy('first_name')\
.agg(f.countDistinct("first_name").alias("distinct_first_name"))\
.orderBy("distinct_first_name", ascending=False)\

distinct_names.limit(5).toPandas()

,first_name,distinct_first_name
0,Aldric,1
1,Guendolen,1
2,Juliette,1
3,Hastie,1
4,Lars,1


In [12]:
total = distinct_names.agg(f.count(distinct_names.first_name)\
                .alias("total_distinct_first_names"))
    
total.toPandas()

,total_distinct_first_names
0,2492


# Query 3 :
Display total sum of purchase ammout from users with job title : ML

In [13]:
dataframe.agg(f.sum(dataframe.purchase_ammount)).toPandas()

,sum(purchase_ammount)
0,1580183


In [14]:
dataframe.where(dataframe.job_title == 'ML').limit(3).toPandas()

,first_name,last_name,email,gender,ip_adress,company_name,purchase_ammount,job_title,user_agent
0,Seth,Frantsev,sfrantsevaw@guardian.co.uk,Male,13.7.163.159,Mybuzz,503,ML,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2..."
1,Issi,Lindstedt,ilindstedt3i@barnesandnoble.com,Female,87.8.250.93,Tanoodle,144,ML,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3..."
2,Dame,Scurfield,dscurfieldik@state.gov,Male,181.37.108.31,Babblestorm,327,ML,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8..."


In [15]:
dataframe.groupBy("job_title")\
.agg(f.sum("purchase_ammount"))\
.where(dataframe.job_title == 'ML').toPandas()

,job_title,sum(purchase_ammount)
0,ML,974


# Query 4 : 

Display the total sum of purchase ammout from all users that use Macintosh

In [16]:
# macintosh_users=dataframe.groupBy("user_agent",'purchase_ammount')\
# .agg(f.sum("purchase_ammount").alias('sum'))\
# .where((dataframe.user_agent.like('%Macintosh%')))\
# .orderBy("sum", ascending=False)\
# .limit(5).toPandas()

In [17]:
all_users = dataframe[['user_agent', 'purchase_ammount']]
all_users.toPandas().head(5)

,user_agent,purchase_ammount
0,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8...",81
1,"""Mozilla/5.0 (Macintosh; U; PPC Mac OS X 10_5_...",789
2,"""Mozilla/5.0 (X11; Linux i686) AppleWebKit/535...",703
3,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",740
4,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",657


In [18]:
mac_users = all_users.filter(all_users.user_agent.like('%Macintosh%'))
mac_users.toPandas().head(5)

,user_agent,purchase_ammount
0,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8...",81
1,"""Mozilla/5.0 (Macintosh; U; PPC Mac OS X 10_5_...",789
2,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_2...",795
3,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_2...",921
4,"""Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_...",518


In [19]:
mac_sum_purchase = mac_users.agg(f.sum(mac_users.purchase_ammount)\
                                    .alias("mac_sum_purchase"))

mac_sum_purchase.toPandas()

,mac_sum_purchase
0,390623


# Query 5 : 

Display the first 10 most used email operators from the female user activity
* Example of email opeartor : yahoo.com

In [20]:
#how many gender there are
dataframe.groupBy("gender")\
.agg(f.countDistinct("gender"))\
.toPandas()

,gender,count(DISTINCT gender)
0,Female,1
1,Male,1


In [21]:
f_users = dataframe[(dataframe.gender=='Female')]
f_users.toPandas().head(5)

,first_name,last_name,email,gender,ip_adress,company_name,purchase_ammount,job_title,user_agent
0,Amandi,Gibbetts,agibbetts0@live.com,Female,39.231.192.239,Brainlounge,81,JCR,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8..."
1,Wilie,Ruf,wruf5@independent.co.uk,Female,55.42.108.22,Flipbug,250,Life Insurance,"""Mozilla/5.0 (Windows; U; Windows NT 6.0; hu-H..."
2,Doretta,Dacey,ddacey6@unc.edu,Female,97.1.82.214,BlogXS,787,HR Strategy,"""Mozilla/5.0 (Windows; U; Windows NT 6.1) Appl..."
3,Kalina,Calow,kcalow8@ezinearticles.com,Female,2.250.236.9,Avaveo,795,Knowledge Management,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_2..."
4,Doe,Grunwald,dgrunwald9@si.edu,Female,186.86.58.133,Shuffledrive,997,Hand-drawn Typography,"""Mozilla/5.0 (X11; Linux i686) AppleWebKit/534..."


In [22]:
email_female = f_users.select("email",f.split("email","@").alias("email_origin"))
email_female.toPandas().head(2)

,email,email_origin
0,agibbetts0@live.com,"[agibbetts0, live.com]"
1,wruf5@independent.co.uk,"[wruf5, independent.co.uk]"


In [23]:
email_operator = email_female.select(email_female.email.alias("email"), email_female.email_origin[1].alias("email_operator"))
email_operator.toPandas().head(5)

,email,email_operator
0,agibbetts0@live.com,live.com
1,wruf5@independent.co.uk,independent.co.uk
2,ddacey6@unc.edu,unc.edu
3,kcalow8@ezinearticles.com,ezinearticles.com
4,dgrunwald9@si.edu,si.edu


In [24]:
email_count = email_operator.groupBy(email_operator.email_operator)\
                                    .agg(f.count(email_operator.email_operator)\
                                    .alias("email_count"))\
                                    .orderBy("email_count", ascending=False)
        
email_count.toPandas().head(10)

,email_operator,email_count
0,github.io,9
1,princeton.edu,8
2,wunderground.com,8
3,slate.com,8
4,moonfruit.com,8
5,soundcloud.com,7
6,simplemachines.org,7
7,ed.gov,7
8,hexun.com,7
9,state.gov,7


# Query 6 : 

Select top 5 companies that have the greatest gender balance in this user activity table

* If same companies have equal gender balance => sort by lexicographic company name order
* Gender balance definition : 
    * The absolute difference between the number of male users and female users
    * Same number of males and females means maximum gender balance


In [25]:
dataframe_q6 = dataframe.select(dataframe.company_name, dataframe.gender,  
                         f.when(dataframe.gender.like('Female'), 1)\
                          .when(dataframe.gender.like('Male'), 0)\
                          .otherwise(-1) \
                          .alias("Col")
                        )

dataframe_q6.toPandas().head(5)

,company_name,gender,Col
0,Brainlounge,Female,1
1,Mydo,Male,0
2,Agivu,Male,0
3,Chatterbridge,Male,0
4,Zoomlounge,Male,0


In [28]:
total_by_gender = dataframe_q6.groupby(dataframe_q6.company_name, dataframe_q6.gender)\
                                .agg(f.count(dataframe_q6.Col)\
                                .alias("gender_count"))\
                                .orderBy("company_name", ascending=False).toPandas()
total_by_gender

,company_name,gender,gender_count
0,Zoozzy,Female,2
1,Zoozzy,Male,2
2,Zooxo,Male,2
3,Zooxo,Female,5
4,Zoovu,Male,2
5,Zoovu,Female,4
6,Zooveo,Female,4
7,Zooveo,Male,4
8,Zoonoodle,Male,3
9,Zoonoodle,Female,3


In [29]:
gender_difference = [0]

for i in range(len(total_by_gender)-1):
    a_company_name = total_by_gender['company_name'][i] 
    a_gender = total_by_gender['gender'][i] 
    a_gender_count = total_by_gender['gender_count'][i]
    
    i = i + 1
    
    b_company_name = total_by_gender['company_name'][i] 
    b_gender = total_by_gender['gender'][i] 
    b_gender_count = total_by_gender['gender_count'][i] 
    
    if a_company_name == b_company_name:
        gender_difference.append(a_gender_count - b_gender_count)
    else:
        gender_difference.append(200)

In [30]:
total_by_gender['gender_difference'] = gender_difference

In [31]:
total_by_gender.head(10)

,company_name,gender,gender_count,gender_difference
0,Zoozzy,Female,2,0
1,Zoozzy,Male,2,0
2,Zooxo,Male,2,200
3,Zooxo,Female,5,-3
4,Zoovu,Male,2,200
5,Zoovu,Female,4,-2
6,Zooveo,Female,4,200
7,Zooveo,Male,4,0
8,Zoonoodle,Male,3,200
9,Zoonoodle,Female,3,0


In [32]:
total_by_gender.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752 entries, 0 to 751
Data columns (total 4 columns):
company_name         752 non-null object
gender               752 non-null object
gender_count         752 non-null int64
gender_difference    752 non-null int64
dtypes: int64(2), object(2)
memory usage: 23.6+ KB


In [33]:
filter1 = total_by_gender.where(total_by_gender.gender_difference<200)

In [35]:
filter2 = filter1.dropna()

In [36]:
dataframe_gender_difference = filter2.tail(-1)

In [37]:
dataframe_gender_difference.head(5)

,company_name,gender,gender_count,gender_difference
1,Zoozzy,Male,2.0,0.0
3,Zooxo,Female,5.0,-3.0
5,Zoovu,Female,4.0,-2.0
7,Zooveo,Male,4.0,0.0
9,Zoonoodle,Female,3.0,0.0


In [38]:
dataframe_gender_difference2 = dataframe_gender_difference[['company_name', 'gender_difference']]
dataframe_gender_difference2.head(5)

,company_name,gender_difference
1,Zoozzy,0.0
3,Zooxo,-3.0
5,Zoovu,-2.0
7,Zooveo,0.0
9,Zoonoodle,0.0


# Query 7: 
Display user first name, user last name where user activity ip address start with '45'

* Example of such ip address : 45.24.32.74

In [39]:
dataframe.toPandas().head(5)

,first_name,last_name,email,gender,ip_adress,company_name,purchase_ammount,job_title,user_agent
0,Amandi,Gibbetts,agibbetts0@live.com,Female,39.231.192.239,Brainlounge,81,JCR,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8..."
1,Mathew,Michin,mmichin1@bloglovin.com,Male,115.35.31.40,Mydo,789,Dell KACE,"""Mozilla/5.0 (Macintosh; U; PPC Mac OS X 10_5_..."
2,Omero,Petrina,opetrina2@ycombinator.com,Male,42.43.198.154,Agivu,703,HP Service Manager,"""Mozilla/5.0 (X11; Linux i686) AppleWebKit/535..."
3,Clevey,Jesse,cjesse3@reverbnation.com,Male,99.233.153.203,Chatterbridge,740,Bioinformatics,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
4,Wit,Gent,wgent4@earthlink.net,Male,235.73.119.127,Zoomlounge,657,Procurement Outsourcing,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537...."


In [43]:
dataframe_q7 = dataframe.filter(dataframe.ip_adress.startswith('45'))\
.select(dataframe.first_name, dataframe.last_name,dataframe.ip_adress).toPandas()
dataframe_q7

,first_name,last_name,ip_adress
0,Gratia,Perrins,45.100.168.26
1,Abby,Parmley,45.56.224.25
2,Gamaliel,Leebetter,45.36.71.18
3,Darius,Rolance,45.192.34.19
4,Ajay,Dymick,45.24.0.223
5,Mark,Cacacie,45.130.81.219
6,Denna,McDade,45.196.113.234
7,Nariko,Bulward,45.248.129.231
8,Fanny,Bush,45.89.113.44
9,Janel,De Few,45.162.230.254


In [44]:
dataframe_q7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
first_name    10 non-null object
last_name     10 non-null object
ip_adress     10 non-null object
dtypes: object(3)
memory usage: 320.0+ bytes


In [45]:
dataframe_q7.dtypes

first_name    object
last_name     object
ip_adress     object
dtype: object